**Get Dataset in Colab From Kaggle**


In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amitkarmakar41","key":"c85bec72c3fbb356b0dac60ae095d7bf"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d shubh0799/churn-modelling

  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 85.0MB/s]


**Code Starts Here**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
dataset = pd.read_csv('/content/churn-modelling.zip')

In [9]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [11]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [12]:
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [13]:
X = pd.concat([X, geography, gender],axis=1)

In [14]:
X = X.drop(['Geography','Gender'],axis=1)

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Age              10000 non-null  int64  
 2   Tenure           10000 non-null  int64  
 3   Balance          10000 non-null  float64
 4   NumOfProducts    10000 non-null  int64  
 5   HasCrCard        10000 non-null  int64  
 6   IsActiveMember   10000 non-null  int64  
 7   EstimatedSalary  10000 non-null  float64
 8   Germany          10000 non-null  uint8  
 9   Spain            10000 non-null  uint8  
 10  Male             10000 non-null  uint8  
dtypes: float64(2), int64(6), uint8(3)
memory usage: 654.4 KB


In [16]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout 

In [20]:
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer= 'he_uniform', activation='relu',input_dim= 11))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 6, kernel_initializer= 'he_uniform', activation='relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))


In [21]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [22]:
classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [23]:
 model_history = classifier.fit(x = X_train, y =y_train, validation_split=0.33, batch_size=10, epochs=100)

Epoch 1/100
536/536 [==============================] - 1s 981us/step - loss: 0.5615 - accuracy: 0.7688 - val_loss: 0.4901 - val_accuracy: 0.7955
Epoch 2/100
536/536 [==============================] - 0s 838us/step - loss: 0.5030 - accuracy: 0.7946 - val_loss: 0.4676 - val_accuracy: 0.7955
Epoch 3/100
536/536 [==============================] - 0s 807us/step - loss: 0.4856 - accuracy: 0.7962 - val_loss: 0.4604 - val_accuracy: 0.7955
Epoch 4/100
536/536 [==============================] - 0s 817us/step - loss: 0.4736 - accuracy: 0.7964 - val_loss: 0.4530 - val_accuracy: 0.7955
Epoch 5/100
536/536 [==============================] - 0s 811us/step - loss: 0.4672 - accuracy: 0.7972 - val_loss: 0.4507 - val_accuracy: 0.7955
Epoch 6/100
536/536 [==============================] - 0s 820us/step - loss: 0.4636 - accuracy: 0.7972 - val_loss: 0.4459 - val_accuracy: 0.7955
Epoch 7/100
536/536 [==============================] - 0s 811us/step - loss: 0.4552 - accuracy: 0.8015 - val_loss: 0.4407 - val_ac

In [24]:
y_pred = classifier.predict(X_test)
y_pred= (y_pred>0.5)

In [25]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [26]:
cm

array([[1583,   12],
       [ 288,  117]])

In [27]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,y_test)
score 

0.85

# Hyper Parameter Tuning

In [45]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid


In [46]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [51]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [52]:
layers = [(20,), (40, 20), (45, 30, 15)]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [54]:
grid_result = grid.fit(X_train, y_train)


In [55]:
print(grid_result.best_score_, grid_result.best_params_)

0.8573749899864197 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}


In [63]:
pred_y = grid.predict(X_test)
y_pred = (pred_y>0.5)

In [64]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_pred,y_test)

In [65]:
cm

array([[1528,   67],
       [ 209,  196]])

In [66]:
score

0.862